In [ ]:
import scipy.stats as stats
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_hac
warnings.filterwarnings("ignore", category=DeprecationWarning)
from google.colab import files
from sklearn.metrics import (roc_auc_score, log_loss, f1_score, precision_recall_curve, auc)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
upload = files.upload()

Saving regime_data.csv to regime_data.csv


In [ ]:
df=pd.read_csv("regime_data.csv")
df['Date']= pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.drop('RV', axis=1)
df = df.rename(columns={'IV': 'VIX'})
df = df.rename(columns={'SKEW': 'IVS'})
df.head()

,VIX,VRP,IVS
Date,,,
1990-01-02,0.1724,0.013732,126.09
1990-01-03,0.1819,0.017098,123.34
1990-01-04,0.1922,0.020951,122.62
1990-01-05,0.2011,0.024846,121.27
1990-01-08,0.2026,0.026346,124.12


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

results = []
daily_data=df.copy()

for col in ['VIX', 'VRP','IVS']:
  for lag in range(23, 45):
      daily_data[f'{col}_lag_{lag}'] = daily_data[col].shift(lag)

daily_data.reset_index(inplace=True)

train_idx, test_idx = train_test_split(daily_data.index, test_size=0.2, shuffle=False)


for max_lag in range(23, 24):  # Lag 23 to 44
    # Create list of features for all lags up to max_lag
    cols = []
    for lag in range(23, max_lag + 1):
        for var in ['VIX', 'IVS', 'VRP']:
            cols.append(f'{var}_lag_{lag}')

    # Define X and y
    X = daily_data[cols]
    y = daily_data['VIX']

    # Clean and align
    X = sm.add_constant(X)
    X, y = X.align(y, join='inner', axis=0)

    # Train/Test split using the pre-defined index
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]

    # Combine and drop any NaNs/Infs
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_df.dropna(inplace=True)

    # Split them back
    X_train = train_df.drop(columns=y_train.name)
    y_train = train_df[y_train.name]

    # Fit OLS model
    model = sm.OLS(y_train, X_train).fit()

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Store summary stats
    results.append({
    'max_lag': max_lag,
    'r2_in_sample': r2_score(y_train, y_pred_train),
    'r2_out_of_sample': r2_score(y_test, y_pred_test),
    'mse_in_sample': mean_squared_error(y_train, y_pred_train),
    'mse_out_of_sample': mean_squared_error(y_test, y_pred_test),
    'aic': model.aic,
    'bic': model.bic
      })

# Convert to DataFrame to view results
results_df = pd.DataFrame(results)
print(model.summary())
results_df

                            OLS Regression Results                            
Dep. Variable:                    VIX   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     4401.
Date:                Tue, 15 Apr 2025   Prob (F-statistic):               0.00
Time:                        20:56:40   Log-Likelihood:                 11668.
No. Observations:                7077   AIC:                        -2.333e+04
Df Residuals:                    7073   BIC:                        -2.330e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0631      0.010      6.493      0.0

,max_lag,r2_in_sample,r2_out_of_sample,mse_in_sample,mse_out_of_sample,aic,bic
0,23,0.651141,0.200659,0.002165,0.00465,-23328.017195,-23300.558774


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

results = []
daily_data=df.copy()

for col in ['VIX', 'VRP','IVS']:
  for lag in range(23, 45):
      daily_data[f'{col}_lag_{lag}'] = daily_data[col].shift(lag)

daily_data.reset_index(inplace=True)

train_idx, test_idx = train_test_split(daily_data.index, test_size=0.2, shuffle=False)


for max_lag in range(23, 24):  # Lag 23 to 44
    # Create list of features for all lags up to max_lag
    cols = []
    for lag in range(23, max_lag + 1):
        for var in ['VIX', 'IVS', 'VRP']:
            cols.append(f'{var}_lag_{lag}')

    # Define X and y
    X = daily_data[cols]
    y = daily_data['VRP']

    # Clean and align
    X = sm.add_constant(X)
    X, y = X.align(y, join='inner', axis=0)

    # Train/Test split using the pre-defined index
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]

    # Combine and drop any NaNs/Infs
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_df.dropna(inplace=True)

    # Split them back
    X_train = train_df.drop(columns=y_train.name)
    y_train = train_df[y_train.name]

    # Fit OLS model
    model = sm.OLS(y_train, X_train).fit()

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Store summary stats
    results.append({
    'max_lag': max_lag,
    'r2_in_sample': r2_score(y_train, y_pred_train),
    'r2_out_of_sample': r2_score(y_test, y_pred_test),
    'mse_in_sample': mean_squared_error(y_train, y_pred_train),
    'mse_out_of_sample': mean_squared_error(y_test, y_pred_test),
    'aic': model.aic,
    'bic': model.bic
      })

# Convert to DataFrame to view results
results_df = pd.DataFrame(results)
print(model.summary())
results_df

                            OLS Regression Results                            
Dep. Variable:                    VRP   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     1386.
Date:                Tue, 15 Apr 2025   Prob (F-statistic):               0.00
Time:                        20:55:56   Log-Likelihood:                 12124.
No. Observations:                7077   AIC:                        -2.424e+04
Df Residuals:                    7073   BIC:                        -2.421e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0613      0.009      6.731      0.0

,max_lag,r2_in_sample,r2_out_of_sample,mse_in_sample,mse_out_of_sample,aic,bic
0,23,0.370227,0.033693,0.001903,0.00886,-24239.188826,-24211.730404


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

results = []
daily_data=df.copy()

for col in ['VIX', 'VRP','IVS']:
  for lag in range(23, 45):
      daily_data[f'{col}_lag_{lag}'] = daily_data[col].shift(lag)

daily_data.reset_index(inplace=True)

train_idx, test_idx = train_test_split(daily_data.index, test_size=0.2, shuffle=False)


for max_lag in range(23, 24):  # Lag 23 to 44
    # Create list of features for all lags up to max_lag
    cols = []
    for lag in range(23, max_lag + 1):
        for var in ['VIX', 'IVS', 'VRP']:
            cols.append(f'{var}_lag_{lag}')

    # Define X and y
    X = daily_data[cols]
    y = daily_data['IVS']

    # Clean and align
    X = sm.add_constant(X)
    X, y = X.align(y, join='inner', axis=0)

    # Train/Test split using the pre-defined index
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]

    # Combine and drop any NaNs/Infs
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_df.dropna(inplace=True)

    # Split them back
    X_train = train_df.drop(columns=y_train.name)
    y_train = train_df[y_train.name]

    # Fit OLS model
    model = sm.OLS(y_train, X_train).fit()

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Store summary stats
    results.append({
    'max_lag': max_lag,
    'r2_in_sample': r2_score(y_train, y_pred_train),
    'r2_out_of_sample': r2_score(y_test, y_pred_test),
    'mse_in_sample': mean_squared_error(y_train, y_pred_train),
    'mse_out_of_sample': mean_squared_error(y_test, y_pred_test),
    'aic': model.aic,
    'bic': model.bic
      })

# Convert to DataFrame to view results
results_df = pd.DataFrame(results)
print(model.summary())
results_df

                            OLS Regression Results                            
Dep. Variable:                    IVS   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     2293.
Date:                Tue, 15 Apr 2025   Prob (F-statistic):               0.00
Time:                        20:56:10   Log-Likelihood:                -21746.
No. Observations:                7077   AIC:                         4.350e+04
Df Residuals:                    7073   BIC:                         4.353e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         38.4795      1.091     35.256      0.0

,max_lag,r2_in_sample,r2_out_of_sample,mse_in_sample,mse_out_of_sample,aic,bic
0,23,0.493051,0.378158,27.319727,114.188643,43499.605306,43527.063727


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

results_all = []

# Loop over each Y candidate (dependent variable)
for y_col in ['VIX', 'IVS', 'VRP']:
    daily_data = df.copy()

    # Create lagged features
    for col in ['VIX', 'IVS', 'VRP']:
        daily_data[f'{col}_lag_23'] = daily_data[col].shift(23)

    # Drop rows with NaNs from lagging
    daily_data.dropna(inplace=True)

    # Time-based train-test split (no shuffle)
    split_idx = int(len(daily_data) * 0.8)
    train = daily_data.iloc[:split_idx]
    test = daily_data.iloc[split_idx:]

    # Define predictors and add constant
    X_train = sm.add_constant(train[['VIX_lag_23', 'IVS_lag_23', 'VRP_lag_23']])
    y_train = train[y_col]
    X_test = sm.add_constant(test[['VIX_lag_23', 'IVS_lag_23', 'VRP_lag_23']])
    y_test = test[y_col]

    # Fit OLS model using training data
    model = sm.OLS(y_train, X_train).fit()

    # Generate predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Error metrics
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)

    # Normalized versions
    norm_mse_train = mse_train / np.var(y_train)
    norm_mse_test = mse_test / np.var(y_test)
    norm_rmse_train = rmse_train / np.mean(y_train)
    norm_rmse_test = rmse_test / np.mean(y_test)

    # Store results
    results_all.append({
        'Target': y_col,
        'R_squared': round(r2_score(y_train, y_train_pred), 2),
        'Normalized_MSE': round(norm_mse_train, 4),
        'AIC': round(model.aic, 2),
        'BIC': round(model.bic, 2)
    })

# Create and display summary dataframe
results_df = pd.DataFrame(results_all)
#print(results_df)
results_df


,Target,R_squared,Normalized_MSE,AIC,BIC
0,VIX,0.65,0.3499,-23322.59,-23295.12
1,IVS,0.49,0.5066,43557.26,43584.72
2,VRP,0.37,0.6299,-24255.73,-24228.27
